In [1]:
import pandas as pd

In [2]:
import os
pn_path = "../../corpus/PN/"

In [3]:
os.listdir(pn_path)

['noun_pn', 'verb_pn']

In [15]:
pd.set_option('display.unicode.east_asian_width', True)
df = pd.read_csv(pn_path+"noun_pn", names=['word', 'label', 'desc'], delimiter="\t")

In [16]:
df

,word,label,desc
0,２，３日,e,〜である・になる（状態）客観
1,10%,e,〜である・になる（状態）客観
2,100%,e,〜である・になる（状態）客観
3,25%,e,〜である・になる（状態）客観
4,80%,e,〜である・になる（状態）客観
...,...,...,...
13309,魏,e,〜に行く（場所）
13310,魑魅魍魎,n,〜である・になる（状態）客観
13311,鮑,e,〜である・になる（状態）客観
13312,鹹味,e,〜である・になる（評価・感情）主観


In [17]:
df['label'].value_counts()

n       4958
e       4954
p       3352
?p?n      35
a          4
?e         4
o          3
?p         2
?p?e       1
　         1
Name: label, dtype: int64

In [18]:
df = df[ (df.label=="p") | (df.label=="n") | (df.label=="e") ]

In [19]:
df['label'].value_counts()

n    4958
e    4954
p    3352
Name: label, dtype: int64

In [22]:
df["label"] = df["label"].replace({'p':1, 'e':0, 'n':-1})

/home/yamada/.local/lib/python3.6/site-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [24]:
df

,word,label,desc
0,２，３日,0,〜である・になる（状態）客観
1,10%,0,〜である・になる（状態）客観
2,100%,0,〜である・になる（状態）客観
3,25%,0,〜である・になる（状態）客観
4,80%,0,〜である・になる（状態）客観
...,...,...,...
13309,魏,0,〜に行く（場所）
13310,魑魅魍魎,-1,〜である・になる（状態）客観
13311,鮑,0,〜である・になる（状態）客観
13312,鹹味,0,〜である・になる（評価・感情）主観


In [25]:
import json
import os
import pandas as pd
import numpy as np
from pathlib import Path
import collections
from sklearn.model_selection import train_test_split
from sklearn import metrics

import sys
sys.path.append("../")
from datatools.analyzer import *
from utterance.error_tools import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [60]:
def make_Xy_pn(df):
    docs = list(nlp.pipe(list( map(str, df.word.values) ), disable=['ner']))
    X_ = [ (doc.vector, doc.text) for doc in docs ]
    X = []
    y = []
    for x, la in zip(X_, df.label):
        if np.linalg.norm(x[0]) == 0:
            continue
        else:
            X.append(x)
            y.append(la)
    
    return X, y

In [52]:
def word2vec_ginza(word):
    if isinstance(word, list):
        docs = list(nlp.pipe( word, disable=['ner']))
        return [ doc.vector for doc in docs ]
    else:
        doc = nlp(word)
        return doc.vector

In [61]:
X, y = make_Xy_pn(df)

In [64]:
X_train_str, X_test_str, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=5)

In [75]:
X_train = [xt[0] for xt in X_train_str]
X_test =  [xt[0] for xt in X_test_str]

In [55]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier()

In [66]:
clf.fit(X_train, y_train)

AdaBoostClassifier()

In [67]:
y_pred = clf.predict(X_test)

In [68]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
print('confusion matrix = \n', confusion_matrix(y_true=y_test, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y_test, y_pred=y_pred))

confusion matrix = 
 [[954 280  90]
 [330 743 285]
 [168 302 463]]
accuracy =  0.5975103734439834


In [76]:
from transformers import BertJapaneseTokenizer, BertForSequenceClassification
import pandas as pd
import numpy as np
import torch

In [77]:
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

In [78]:
sent = "吾輩は猫である。名前はまだない。"
tokenized_text = tokenizer.tokenize(sent)

In [79]:
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

In [80]:
indexed_tokens

[7184, 30046, 9, 6040, 12, 31, 8, 1381, 9, 2941, 80, 8]

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    "cl-tohoku/bert-base-japanese-whole-word-masking", # 日本語Pre trainedモデルの指定
    num_labels = 2, # ラベル数（今回はBinayなので2、数値を増やせばマルチラベルも対応可）
    output_attentions = False, # アテンションベクトルを出力するか
    output_hidden_states = True, # 隠れ層を出力するか
)

In [81]:
from gensim.models import KeyedVectors

w2v_path = "../../corpus/w2v/"
w2v_model = KeyedVectors.load_word2vec_format(w2v_path+ "dep-ja-300dim")

In [84]:
w2v_model.most_similar("死者")

[('負傷者', 0.8886025547981262),
 ('犠牲者', 0.8725400567054749),
 ('逮捕者', 0.8267808556556702),
 ('日本兵', 0.785821795463562),
 ('生存者', 0.7841004133224487),
 ('戦没者', 0.7818393707275391),
 ('失業者', 0.7770141959190369),
 ('訪問者', 0.7766074538230896),
 ('感染者', 0.765667200088501),
 ('米兵', 0.7612189054489136)]

In [90]:
w2v_model.similarity("良い", "我慢")

0.4483085